# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input
from tensorflow.keras.callbacks import EarlyStopping
import optuna
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima

#SEED   
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv('Data/london_merged.csv')
display(df.head(2))

- Metadata:
  - "timestamp" - timestamp field for grouping the data
  - "cnt" - the count of a new bike shares
  - "t1" - real temperature in C
  - "t2" - temperature in C "feels like"
  - "hum" - humidity in percentage
  - "wind_speed" - wind speed in km/h
  - "weather_code" - category of the weather
  - "is_holiday" - boolean field - 1 holiday / 0 non holiday
  - "is_weekend" - boolean field - 1 if the day is weekend
  - "season" - category field meteorological seasons: 0-spring ; 1-summer; 2-fall; 3-winter.
  - "weathe_code" category description:
     - 1 = Clear ; mostly clear but have some values with haze/fog/patches of fog/ fog in vicinity 
     - 2 = scattered clouds / few clouds 
     - 3 = Broken clouds 
     - 4 = Cloudy 
     - 7 = Rain/ light Rain shower/ Light rain 
     - 10 = rain with thunderstorm 
     - 26 = snowfall 
     - 94 = Freezing Fog

In [ ]:
df.info()

# Data Wrangling

In [ ]:
#Convert the timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
#Sort the values by timestamp
df = df.sort_values('timestamp')

In [ ]:
#Missing values
df.isnull().sum()

- No missing values. But there might be missing timestamps.

In [ ]:
#Check for missing timestamps
all_days = pd.date_range(start=df['timestamp'].min(), end=df['timestamp'].max(), freq='h')
missing_days = all_days[~all_days.isin(df['timestamp'])]
print('Number of missing timestamps:', len(missing_days))

In [ ]:
missing_days[0]

- 130 timestamps are missing. We will imput them using existing values.

In [ ]:
#London holidays
import holidays
uk_holidays = holidays.UK(years=[df['timestamp'].dt.year.min(), df['timestamp'].dt.year.max()])
uk_holidays

In [ ]:
#Create new dataframe using all days
df_full = pd.DataFrame(all_days, columns=['timestamp'])
#Merge with df to get cnt, t1, t2, hum, wind_speed, weather_code, season
df_full = df_full.merge(df[['timestamp', 'cnt', 't1', 't2', 'hum', 'wind_speed', 'weather_code', 'season']], on='timestamp', how='left')
#is_holiday column: 1 if holiday, 0 if not
df_full['is_holiday'] = np.where(df_full['timestamp'].dt.date.isin(uk_holidays), 1, 0)
df_full['is_weekend'] = np.where(df_full['timestamp'].dt.dayofweek.isin([5, 6]), 1, 0)

#Backfill missing values
df_full = df_full.ffill()
df = df_full.copy()

In [ ]:
df.isnull().sum()

In [ ]:
missing_days = all_days[~all_days.isin(df['timestamp'])]
print('Number of missing timestamps:', len(missing_days))

In [ ]:
#Set the timestamp as the index
df.set_index('timestamp', inplace=True)
#Set period to 1 hour
df.index = pd.DatetimeIndex(df.index).to_period('h')
df.head(2)

In [ ]:
#Remove duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# Boxplot of all the columns
plt.figure(figsize=(10, 12))
cols = df.columns
for i in range(1, len(cols)):
    plt.subplot(3, 3, i)
    sns.boxplot(df[cols[i]])
    plt.title(cols[i])
plt.tight_layout()
plt.show()

- There is no abnormal data in the dataset.

In [ ]:
# Correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

- Real and feels like temperature are highly correlated. Let's use feels like temperature since it is more likely to impact the decision.

In [ ]:
#Drop t1
df.drop('t1', axis=1, inplace=True)

In [ ]:
#Map codes
#Map weather code:
weather_desc = {
    1: 'Clear', 2: 'Scattered_Clouds', 3: 'Broken_Clouds', 4: 'Cloudy', 7: 'Rain', 10: 'Storm', 26: 'Snowfall', 94: 'Freezing_Fog'
}
df['weather_code'] = df['weather_code'].map(weather_desc)

#Map is_holiday:
df['is_holiday'] = df['is_holiday'].map({0:'No_Holiday', 1:'Holiday'})

#Map is_weekend:
df['is_weekend'] = df['is_weekend'].map({0:'Weekday', 1:'Weekend'})

#Map season:
seasons = {0:'Spring', 1:'Summer', 2:'Fall', 3:'Winter'}
df['season'] = df['season'].map(seasons)
df.head(2)

In [ ]:
#One hot encoding for categorical variables
df = pd.get_dummies(df, drop_first=True, dtype=int)
df.head(2)

In [ ]:
#Split the data into train and test
X = df.drop('cnt', axis=1)
y = df['cnt']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=False)

#Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Models

## SARIMAX

### Check Stationarity

In [ ]:
#Plot to check for stationarity
plt.figure(figsize=(15, 6))
plt.plot(y_train.to_timestamp())
plt.title('Hourly Bike Rentals')
plt.show()

- There is no noticeable trend in the data. The data looks stationary. Let's verify this with the Augmented Dickey-Fuller test.

In [ ]:
def check_stationarity(data):
    print('Null Hypothesis: Presence of unit root (Data is not stationary)')
    print('Alternate Hypothesis: Absence of unit root (Data is stationary)')
    result = adfuller(data, autolag='AIC')
    print(result)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    if result[1] > 0.05:
        print('Data is not stationary')
    else:
        print('Data is stationary')

#Check stationarity of the target variable
check_stationarity(y_train)

In [ ]:
type(y_train.to_timestamp())

In [ ]:
#ACF and PACF plots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.figure(figsize=(20, 15))
plt.subplot(211)
plot_acf(y_train, lags=168, ax=plt.gca())
plt.subplot(212)
plot_pacf(y_train, lags=168, ax=plt.gca())
plt.tight_layout()
plt.show()

- Estimating AR terms: The PACF plot shows a significant spike at lag 1, 2, 3, 4, 5 and 6. Therefore, p = 6.
- Estimating I term: The ADF test shows that the data is already stationary. Therefore, d = 0.
- Estimating MA terms: The ACF plot shows a significant spike at lag 1, 2, 3, and 4. Therefore, q = 4.
- Estimating seasonal AR terms: The PACF plot shows a significant spike at lag 24. Therefore, P = 1.
- Estimating seasonal I term: The ADF test shows that the data is already stationary. Therefore, D = 0.
- Estimating seasonal MA terms: The ACF plot shows a significant spike at eight 24 lag intervals. Let's start Q with 8.
- ACF plot shows strong correlation at lag 24. This indicates a daily seasonality. Therefore, s = 24.

However, running the model with these parameters is computationally expensive. For this reason, as an example, I will use small order values.

# LSTM

In [ ]:
#LSTM model with lookback of 24 hours*7 days
lookup_hours = 24*7
forecast_horizon = 24
def create_dataset(X, y, lookback):
    Xs, ys = [], []
    for i in range(len(X) - lookback - forecast_horizon):
        Xs.append(X[i:(i + lookback)])
        ys.append(y[i + lookback:i + lookback + forecast_horizon])
    return np.array(Xs), np.array(ys)

X_train, y_train = create_dataset(X_train, y_train, lookup_hours)
X_test, y_test = create_dataset(X_test, y_test, lookup_hours)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input
from keras.callbacks import EarlyStopping
import numpy as np
import optuna

# Tuning the LSTM model using Optuna
def objective(trial):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(units=trial.suggest_int('units', 32, 512)))
    model.add(Dropout(trial.suggest_float('dropout', 0.2, 0.5)))
    model.add(Dense(forecast_horizon))
    model.compile(optimizer='adam', loss='mse')
    
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, 
                        callbacks=[EarlyStopping('val_loss', patience=5)], verbose=0)
    
    return np.min(history.history['val_loss'])

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10, show_progress_bar=True)
trial = study.best_trial
print('Loss: ', trial.value)
print('Best hyperparameters: ', trial.params)
